# Este es el proyecto final de la ruta Data Science de IBM y Coursera
En este proyecto se deberá poner a prueba lo aprendido durante toda la ruta de aprendizaje

In [1]:
import pandas as pd
import numpy as np

In [2]:
# La seguna instruccion dice:  Imprima la siguiente declaración: ¡Hola, Proyecto Final del Curso!, pero no indica si con python o markdown, asi que lo he hecho de las dos formas
print("¡Hola, Proyecto Final del Curso!")

¡Hola, Proyecto Final del Curso!


# Agrupación y Segmentación de Vecindarios en la Ciudad de Toronto, Canadá

## Parte 1 - Extraccion de los datos

### Librerias necesarias

In [13]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import lxml
import lxml.etree
import html5lib

### Creando el dataframe desde Wikipedia

In [14]:


url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
data = pd.read_html(url)
df = data[0]
df.head()



,0,1,2,3,4,5,6,7,8
0,M1ANot assigned,M2ANot assigned,M3ANorth York(Parkwoods),M4ANorth York(Victoria Village),M5ADowntown Toronto(Regent Park / Harbourfront),M6ANorth York(Lawrence Manor / Lawrence Heights),M7AQueen's Park(Ontario Provincial Government),M8ANot assigned,M9AEtobicoke(Islington Avenue)
1,M1BScarborough(Malvern / Rouge),M2BNot assigned,M3BNorth York(Don Mills)North,M4BEast York(Parkview Hill / Woodbine Gardens),"M5BDowntown Toronto(Garden District, Ryerson)",M6BNorth York(Glencairn),M7BNot assigned,M8BNot assigned,M9BEtobicoke(West Deane Park / Princess Garden...
2,M1CScarborough(Rouge Hill / Port Union / Highl...,M2CNot assigned,M3CNorth York(Don Mills)South(Flemingdon Park),M4CEast York(Woodbine Heights),M5CDowntown Toronto(St. James Town),M6CYork(Humewood-Cedarvale),M7CNot assigned,M8CNot assigned,M9CEtobicoke(Eringate / Bloordale Gardens / Ol...
3,M1EScarborough(Guildwood / Morningside / West ...,M2ENot assigned,M3ENot assigned,M4EEast Toronto(The Beaches),M5EDowntown Toronto(Berczy Park),M6EYork(Caledonia-Fairbanks),M7ENot assigned,M8ENot assigned,M9ENot assigned
4,M1GScarborough(Woburn),M2GNot assigned,M3GNot assigned,M4GEast York(Leaside),M5GDowntown Toronto(Central Bay Street),M6GDowntown Toronto(Christie),M7GNot assigned,M8GNot assigned,M9GNot assigned


### Filtrando el dataframe

In [15]:
filter_df = df.loc[ df['Borough'] != 'Not assigned']
print(f"Not assigned Boroughs: {(filter_df.Borough == 'Not assigned').sum()}")

KeyError: 'Borough'

In [16]:
print(f"Not assigned Neighbourhoods: {(filter_df.Neighbourhood == 'Not assigned').sum()}")

NameError: name 'filter_df' is not defined

In [17]:
filter_df.reset_index(drop=True, inplace=True)
filter_df.head()

NameError: name 'filter_df' is not defined

In [18]:
print(f"Shape of filtered Dataframe: {filter_df.shape}")

NameError: name 'filter_df' is not defined

## Parte 2: Gesopatial data

In [19]:
!pip install geocoder --quiet
import geocoder
from geopy.geocoders import Nominatim

### Usando codigos postales para latitud y longitud

In [21]:
#define empty lists for geospatial values and None values
lat = []
long = []
nones = []

#initialize geolocator
geolocator = Nominatim(user_agent = 'toronto_zipcodes_M')

#loop through filter_df['Postal Code'] values and append to the respective lists
for code in filter_df['Postal Code']:
    location = geolocator.geocode(code)
    if location is None:
        nones.append(None)
        nones.append(None)
    else:
        latitude = location.latitude
        longitude = location.longitude
    lat.append(latitude)
    long.append(longitude)

NameError: name 'filter_df' is not defined

In [22]:
print(f"Total non-None values: {len(lat)}, {len(long)}")
print(f"Total None values: {len(nones)}")

Total non-None values: 0, 0
Total None values: 0


### Agregando valores de latitud y longitud

In [23]:
filter_df['Latitude'] = lat
filter_df['Longitude'] = long
filter_df.head()

NameError: name 'filter_df' is not defined

## Parte 3: Explorando y agrupando vecinos

### Librerias

In [24]:
import re
import folium
import requests

### Extrayendo Borough 

In [ ]:


string = filter_df['Borough'].values
regex = r"\w*\sToronto"

borough = []
none_vals = []
for row in string:
    val = re.findall(regex, row)
    if val == []:
        none_vals.append(val)
    else:
        borough.append(val)



### Explorando dataframe

In [ ]:
explore_df = filter_df.loc[filter_df['Borough'].isin(chosen_boroughs)].reset_index(drop=True)

#change of spelling from 'Neighbourhood' to 'Neighborhood'
cols = {'Postal Code':'Postal Code',  'Borough':'Borough', 'Neighbourhood':'Neighborhood', 'Latitude':'Latitude', 'Longitude':'Longitude'}
explore_df.rename(columns = cols, inplace=True)
explore_df.head()

In [26]:
print(f"Total unique Boroughs: {explore_df.Borough.nunique()}\nTotal unique Neighborhoods: {explore_df.Neighborhood.nunique()}")

NameError: name 'explore_df' is not defined

### Mapeando vecindarios

In [27]:


address = 'Toronto, Ontario, Canada'
geolocator = Nominatim(user_agent='Toronto_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(f"Coordinates of Toronto are: {latitude}, {longitude}")



Coordinates of Toronto are: 43.6534817, -79.3839347


In [28]:


toronto_map = folium.Map(location= [latitude, longitude], zoom_start=10)

#add Neighborhood markers on toronto_map
for lat, long, bor, hood in zip(explore_df['Latitude'], explore_df['Longitude'],explore_df['Borough'],explore_df['Neighborhood']):
    label = folium.Popup('{}, {}'.format(bor, hood), parse_html=True)
    folium.CircleMarker([lat, long],
        radius=4,
        popup=label,
        color='cyan',
        fill=True,
        fill_color='blue',
        fill_opacity=0.6,
        parse_html=False).add_to(toronto_map)

toronto_map



NameError: name 'explore_df' is not defined